In [29]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score , recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler


In [30]:
df = pd.read_csv('df_para_modelo.csv')

In [31]:
df

,Available Extra Rooms in Hospital,staff_available,Severity of Illness,Visitors with Patient,Insurance,Admission_Deposit,Ward_Facility_Code_A,Ward_Facility_Code_B,Ward_Facility_Code_C,Ward_Facility_Code_D,...,Age_41-50,Age_51-60,Age_61-70,Age_71-80,Age_81-90,Age_91-100,Type of Admission_Emergency,Type of Admission_Trauma,Type of Admission_Urgent,estadia_larga
0,4,0,0,4,1,2966.408696,0,0,0,1,...,1,0,0,0,0,0,0,1,0,0
1,4,2,2,2,0,3554.835677,0,1,0,0,...,0,0,0,0,0,0,0,1,0,1
2,2,8,0,2,1,5624.733654,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,7,1,4,0,4814.149231,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
4,2,10,1,2,0,5169.269637,0,0,0,0,...,0,0,0,1,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409995,1,1,2,3,1,3902.291076,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
409996,5,5,0,4,0,4771.995223,0,0,0,1,...,0,0,0,0,0,0,0,1,0,1
409997,4,9,2,4,1,3816.994210,0,0,0,1,...,0,0,0,0,0,0,1,0,0,1
409998,10,10,1,2,1,3841.577491,0,0,0,1,...,0,0,0,0,0,0,0,1,0,1


In [4]:
X=df.iloc[:,:-1]
y=df.iloc[:,-1]

## Estandarizacion y  Reduccion de dimencion con PCA

In [5]:
#normalizamos los datos
scaler=StandardScaler()
scaler.fit(X) # calculo la media para poder hacer la transformacion
X_scaled=scaler.transform(X)# Ahora si, escalo los datos y los normalizo

from sklearn.decomposition import PCA

pca = PCA(0.98)
X_reduc = pca.fit_transform(X_scaled) 

In [8]:
# print("shape of X_pca", X_reduc.shape)
# expl = pca.explained_variance_ratio_
# print(expl)
# print('suma:',sum(expl[0:9]))

# pca = PCA(n_components=9)
# X_nuevo = pca.fit_transform(X_scaled)

shape of X_pca (410000, 14)
[3.34641131e-01 1.01525363e-01 9.39206702e-02 9.13086538e-02
 8.19585054e-02 7.88846616e-02 7.74213481e-02 7.03783135e-02
 5.33083083e-02 1.66530453e-02 7.67816717e-26 3.91237654e-26
 1.45518692e-26 4.18689098e-28]
suma: 0.9833469546804391


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_reduc, y, test_size=0.30, random_state=42)

### Benchmark

In [23]:
print('El benchmark es:',y.sum()/y.count())

El benchmark es: 0.6229658536585366


Veo si se mantiene la distribucion

In [24]:
print('El benchmark es:',y_train.sum()/y_train.count())
print('El benchmark es:',y_test.sum()/y_test.count())

El benchmark es: 0.6232508710801393
El benchmark es: 0.6223008130081301


### Decision Tree

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(random_state=42) 
model = GridSearchCV(cv=5, error_score= np.nan,
             estimator=clf,
             param_grid={'ccp_alpha': [0.1, 0.01, 0.001],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': [5, 6, 7, 8, 9],
                         'max_features': ['auto', 'sqrt', 'log2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=True)

In [11]:
model.fit(X_train, y_train)

Fitting 5 folds for each of 90 candidates, totalling 450 fits


c:\Users\Carlos\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\tree\_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
c:\Users\Carlos\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\tree\_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
c:\Users\Carlos\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\tree\_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
c:\Users\Carlos\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\tree\_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'ccp_alpha': [0.1, 0.01, 0.001],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': [5, 6, 7, 8, 9],
                         'max_features': ['auto', 'sqrt', 'log2']},
             verbose=True)

In [12]:
model.best_estimator_

DecisionTreeClassifier(ccp_alpha=0.001, criterion='entropy', max_depth=8,
                       max_features='log2', random_state=42)

In [13]:
print("Mejores hiperparámetros: "+str(model.best_params_))
print("Mejor Score: "+str(model.best_score_)+'\n')

Mejores hiperparámetros: {'ccp_alpha': 0.001, 'criterion': 'entropy', 'max_depth': 8, 'max_features': 'log2'}
Mejor Score: 0.669020905923345



In [14]:
tree = DecisionTreeClassifier(ccp_alpha=0.001, criterion='entropy', max_depth=8, max_features='log2', random_state=42)

In [15]:
tree.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.001, criterion='entropy', max_depth=8,
                       max_features='log2', random_state=42)

In [18]:
y_pred = tree.predict(X_test)
print(accuracy_score(y_test,y_pred))
precision = recall_score(y_test, y_pred, average= None)
print('recall:',precision)

0.6661138211382114
recall: [0.3423596  0.86261317]


## Booosting

In [19]:
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier

In [20]:
clf_xgb = xgb.XGBClassifier(n_estimators=150,
                            seed=42, use_label_encoder= False)
                            
# Entrenamos
clf_xgb.fit(X_train, y_train)

c:\Users\Carlos\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=150,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=42, ...)

In [26]:
y_train_pred = clf_xgb.predict(X_train)
y_test_pred = clf_xgb.predict(X_test)

# Evaluamos
print('Accuracy en train:',accuracy_score(y_train, y_train_pred))
print('Accuracy en test:', accuracy_score(y_test, y_test_pred))
precision = recall_score(y_test, y_test_pred, average= None)
print('Recall :',precision)

Accuracy en train: 0.7652961672473868
Accuracy en test: 0.7128617886178862
Recall : [0.52254773 0.82837098]


## SVM

## Bagging

In [19]:
from sklearn.ensemble import BaggingClassifier

In [20]:
bagging = BaggingClassifier(base_estimator=DecisionTreeClassifier(), bootstrap_features=False, n_estimators= 100, n_jobs = -1)
bagging.fit(X_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100,
                  n_jobs=-1)

In [21]:

y_train_pred = bagging.predict(X_train)
y_test_pred = bagging.predict(X_test)
print(accuracy_score(y_train, y_train_pred))
print(accuracy_score(y_test, y_test_pred))
precision = recall_score(y_test, y_test_pred, average= None)
print(precision)

0.9999930313588851
0.7273170731707317
[0.62468519 0.78960846]


## KMeans

In [25]:
from sklearn.cluster import KMeans

kmeans_1 = KMeans(n_clusters=2, random_state=0)
kmeans_1.fit(X_train)

KMeans(n_clusters=2, random_state=0)

### val

In [27]:
y_train_pred = kmeans_1.predict(X_train)
y_test_pred = kmeans_1.predict(X_test)

# Evaluamos
print('Accuracy en train:', accuracy_score(y_train, y_train_pred))
print('Accuracy en test:', accuracy_score(y_test, y_test_pred))
precision = recall_score(y_test, y_pred, average= None)
print('recall:',precision)

Accuracy en train: 0.44237979094076657
Accuracy en test: 0.44239024390243903
recall: [0.3423596  0.86261317]


## Test

In [36]:
df_test = pd.read_csv('df_trans_test.csv')

In [ ]:
def crear_pred(modelo):
    scaler=StandardScaler()
    scaler.fit(df_test) # calculo la media para poder hacer la transformacion
    X_to_pred=scaler.transform(df_test)# Ahora si, escalo los datos y los normalizo
    
    pca = PCA(0.98)
    X_df_test = pca.fit_transform(X_to_pred)   

    pred = modelo.predict(X_df_test)
    pred = pd.DataFrame({'pred': pred})

    pred.to_csv('Heizhem.csv',index=False)

In [ ]:
crear_pred()